In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator         #For data augmentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tkinter as tk
from tkinter import filedialog                                              #Provides a dialog box to select files
from PIL import Image, ImageTk                                              #Converts images for display in Tkinter Files
import numpy as np


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.45G/1.45G [00:56<00:00, 27.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1


# Step 2: Loading and Preprocessing the Data

In [10]:
#Set Up Directories
train_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train"
valid_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid"
test_dir =r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test"

#set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


# Step 3: Building the CNN Model

In [15]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout     # Import Dropout

#Building the CNN Model
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0,5),
  Dense(1, activation='sigmoid')                                                      #Bianary Classification Wildfire or No Wildfire
])

#Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Step 4: Training the Model

In [18]:
#Train the Model
history = model.fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 249s 260ms/step - accuracy: 0.8717 - loss: 0.2857 - val_accuracy: 0.9429 - val_loss: 0.1643
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 253s 251ms/step - accuracy: 0.9326 - loss: 0.1782 - val_accuracy: 0.9286 - val_loss: 0.1821
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 241s 255ms/step - accuracy: 0.9406 - loss: 0.1549 - val_accuracy: 0.9490 - val_loss: 0.1257
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 259s 251ms/step - accuracy: 0.9508 - loss: 0.1318 - val_accuracy: 0.9441 - val_loss: 0.1421
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 243s 257ms/step - accuracy: 0.9560 - loss: 0.1187 - val_accuracy: 0.9559 - val_loss: 0.1169
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 256s 251ms/step - accuracy: 0.9576 - loss: 0.1118 - val_accuracy: 0.9589 - val_loss: 0.1127
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 263s 253ms/step - accuracy: 0.9649 - loss: 0.0955 - val_accuracy: 0.9549 - val_loss: 0.1236
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 251s 265ms/step - accuracy: 0.9690 - loss: 0.08

In [19]:
#save the model
model.save('wildfire_model.h5')

# Step 5: Building the GUI with tinker

In [20]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('wildfire_model.h5')

In [22]:
#Load the Model Again
model = load_model('wildfire_model.h5')
print("Model Loaded Successfully")

Model Loaded Successfully


In [23]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

TclError: no display name and no $DISPLAY environment variable

In [24]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [25]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('wildfire_model.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated input definition
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains a wildfire."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27df765ff925d41af4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
